Laptop price prediction based on specification
https://www.kaggle.com/datasets/muhammetvarl/laptop-price/data

In [224]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from math import sqrt

In [225]:
df = pd.read_csv("/Users/kirillsobolev/Documents/GitHub/Introduction-to-ML/exercise1/laptop_price.csv", encoding="latin-1")
# data set was not working with default utf encoding
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


Step 1. Check out missing values

In [226]:
df.isna().sum()

laptop_ID           0
Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price_euros         0
dtype: int64

Step 2. Get rid of not related columns.

In [227]:
# Closer look on Product column, looks like it is just a full name of laptop
df["Product"].unique()

array(['MacBook Pro', 'Macbook Air', '250 G6', 'Aspire 3',
       'ZenBook UX430UN', 'Swift 3', 'Inspiron 3567', 'MacBook 12"',
       'IdeaPad 320-15IKB', 'XPS 13', 'Vivobook E200HA',
       'Legion Y520-15IKBN', '255 G6', 'Inspiron 5379',
       '15-BS101nv (i7-8550U/8GB/256GB/FHD/W10)', 'MacBook Air',
       'Inspiron 5570', 'Latitude 5590', 'ProBook 470', 'LapBook 15.6"',
       'E402WA-GA010T (E2-6110/2GB/32GB/W10)',
       '17-ak001nv (A6-9220/4GB/500GB/Radeon', 'IdeaPad 120S-14IAP',
       'Inspiron 5770', 'ProBook 450',
       'X540UA-DM186 (i3-6006U/4GB/1TB/FHD/Linux)', 'Inspiron 7577',
       'X542UQ-GO005 (i5-7200U/8GB/1TB/GeForce', 'Aspire A515-51G',
       'Inspiron 7773', 'IdeaPad 320-15ISK', 'Rog Strix',
       'X751NV-TY001T (N4200/4GB/1TB/GeForce', 'Yoga Book', 'ProBook 430',
       'Inspiron 3576', '15-bs002nv (i3-6006U/4GB/128GB/FHD/W10)',
       'VivoBook Max', 'GS73VR 7RG',
       'X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux)', 'Vostro 5471',
       'IdeaPad 520S-1

In [228]:
# Delete product and laptop id columns.
df = df.drop(["laptop_ID", "Product"], axis=1)

Step 3. In column weight and RAM there are kilograms and gigabytes, let's get rid of them and leave just a number

In [229]:
# check that there is only GB 
df["Ram"].unique()

array(['8GB', '16GB', '4GB', '2GB', '12GB', '6GB', '32GB', '24GB', '64GB'],
      dtype=object)

In [230]:
# extract only numeric value from Ram column
df['Ram'] = df['Ram'].str[:-2].astype(int)

In [231]:
# same for weight, but type has to be float
df["Weight"] = df["Weight"].str[:-2].astype(float)

In [232]:
df.head()

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60


Step 4 let's deal with all text columns.
1. Company - I think there might be few big companies from which comes biggest part of laptops and some small companies which we could join to some kind of "Other company" category
2. TypeName - feels like there should not be so much categories.
3. OpSys - same as company

In [233]:
df["Company"].value_counts()

Dell         297
Lenovo       297
HP           274
Asus         158
Acer         103
MSI           54
Toshiba       48
Apple         21
Samsung        9
Razer          7
Mediacom       7
Microsoft      6
Xiaomi         4
Vero           4
Chuwi          3
Google         3
Fujitsu        3
LG             3
Huawei         2
Name: Company, dtype: int64

In [234]:
# Let's use one hot encoder to create categorical columns with numeric values
variables = ["Company"]

encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
one_hot_encoded = encoder.fit_transform(df[variables]).astype(int)
df = pd.concat([df,one_hot_encoded],axis=1).drop(columns=variables)

In [235]:
# After that we can drop columns with companies which are not so presented in dataset. Basically it will join all of them to "other" category. 
df = df.drop(["Company_Samsung","Company_Razer","Company_Mediacom","Company_Microsoft","Company_Xiaomi","Company_Vero",
              "Company_Chuwi","Company_Google","Company_Fujitsu","Company_LG","Company_Huawei"], axis=1)

In [236]:
# Next step is dealing with OS column.
df["OpSys"].value_counts()

Windows 10      1072
No OS             66
Linux             62
Windows 7         45
Chrome OS         27
macOS             13
Mac OS X           8
Windows 10 S       8
Android            2
Name: OpSys, dtype: int64

In [237]:
# In my opinion, we should divide OS into 2 categories only, Windows and Other. Because only windows here is actually comercial OS and it might affect the price.
# Next code searches for the keyword 'Windows' and changing value to 1 if finds it and to 0 if not
df['OpSys'] = df['OpSys'].str.contains('Windows', 
                                        case=False, regex=True).astype(int)

In [238]:
# Now laptop type 
df["TypeName"].value_counts()

Notebook              727
Gaming                205
Ultrabook             196
2 in 1 Convertible    121
Workstation            29
Netbook                25
Name: TypeName, dtype: int64

In [239]:
# Let's again use encoder

variables = ["TypeName"]

encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
one_hot_encoded = encoder.fit_transform(df[variables]).astype(int)
df = pd.concat([df,one_hot_encoded],axis=1).drop(columns=variables)

In [240]:
# And delete last column to optimize model a little
df = df.drop(["TypeName_Netbook"], axis=1)

In [241]:
df.head()

,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Company_Acer,...,Company_Dell,Company_HP,Company_Lenovo,Company_MSI,Company_Toshiba,TypeName_2 in 1 Convertible,TypeName_Gaming,TypeName_Notebook,TypeName_Ultrabook,TypeName_Workstation
0,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,0,1.37,1339.69,0,...,0,0,0,0,0,0,0,0,1,0
1,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,0,1.34,898.94,0,...,0,0,0,0,0,0,0,0,1,0
2,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,0,1.86,575.00,0,...,0,1,0,0,0,0,0,1,0,0
3,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,0,1.83,2537.45,0,...,0,0,0,0,0,0,0,0,1,0
4,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,0,1.37,1803.60,0,...,0,0,0,0,0,0,0,0,1,0


We are left with only 4 problematic columns.
1. Screen Resolution
2. CPU
3. Memory
4. GPU

Screen resolution. There are too many different names in this dataset to use encoder, but we can extract some values out of it.

We are able to extract pixels amount and we do have Inches data already. That means that we can coint PPI. Pixels Per Inch (PPI) is a measure of Image resolution, the number of individual square pixels per a linear inch within a digital image. 

To count the pixels per inch (PPI) of a monitor when you have its size in inches (the diagonal length of the screen) and its resolution in the format width x height (the number of horizontal and vertical pixels)

The other thing we can find out from this columns is if laptop has touchscreen or not

In [242]:
#Extracting only resolution from ScreenResolution column
df['PPI'] = df['ScreenResolution'].str.extract(r'(\d+x\d+)')
#Creating Width and Height to calculate PPI
df[['Width', 'Height']] = df['PPI'].str.split('x', expand=True).astype('int')
# Finally, count PPI
df['PPI'] = (((df['Width']**2 + df['Height']**2)**0.5)/df['Inches']).round(2)

In [243]:
#Creating Touchscreen column that based on the presence of the term 'Touchscreen'
df['Touchscreen'] = df['ScreenResolution'].str.contains('Touchscreen', 
                                        case=False, regex=True).astype(int)

In [244]:
# Now we can finally drop Resoultion, width, height and inches columns. They all are basically present same information as PPI
df = df.drop(["Width", "Height", "Inches", "ScreenResolution"], axis=1)


Out of the CPU column we can get cpu frequncy value and manufacturer's name.

In [245]:
#Extracting manufacturer name 
df['Cpu_Manufacturer'] = df['Cpu'].str.split(' ').str[0]
#Extracting Ghz number
df['Cpu_Ghz'] = df['Cpu'].str.extract(r'(\d+(\.\d+)?)GHz')[0].astype(float)

In [246]:
df['Cpu_Manufacturer'].value_counts()

Intel      1240
AMD          62
Samsung       1
Name: Cpu_Manufacturer, dtype: int64

In [248]:
#Samsung is definetly otlier here, so let's get rid of it
df = df[df['Cpu_Manufacturer'] != 'Samsung']
# as well as cpu column because we don't need it anymore
df = df.drop(['Cpu'], axis=1)

Memory. From here we can obtain memory type and memory capacity


In [249]:
# Looks like there are 4 types of memory: Flash, SSD, HDD, Hybrid. Also looks like some of laptops have two different storages.
# Another problem is that we have terrabytes and gigabytes in same column, as well as numbers sometimes are represented as integers (1) or floats (1.0)
df['Memory'].unique()

array(['128GB SSD', '128GB Flash Storage', '256GB SSD', '512GB SSD',
       '500GB HDD', '256GB Flash Storage', '1TB HDD',
       '32GB Flash Storage', '128GB SSD +  1TB HDD',
       '256GB SSD +  256GB SSD', '64GB Flash Storage',
       '256GB SSD +  1TB HDD', '256GB SSD +  2TB HDD', '32GB SSD',
       '2TB HDD', '64GB SSD', '1.0TB Hybrid', '512GB SSD +  1TB HDD',
       '1TB SSD', '256GB SSD +  500GB HDD', '128GB SSD +  2TB HDD',
       '512GB SSD +  512GB SSD', '16GB SSD', '16GB Flash Storage',
       '512GB SSD +  256GB SSD', '512GB SSD +  2TB HDD',
       '64GB Flash Storage +  1TB HDD', '180GB SSD', '1TB HDD +  1TB HDD',
       '32GB HDD', '1TB SSD +  1TB HDD', '512GB Flash Storage',
       '128GB HDD', '240GB SSD', '8GB SSD', '508GB Hybrid', '1.0TB HDD',
       '512GB SSD +  1.0TB Hybrid', '256GB SSD +  1.0TB Hybrid'],
      dtype=object)

In [ ]:
#TODO разделить жесткие диски по типу хранилища и размеру данных. Поделить гпу на производителя и размер памяти 